Question: What topics are being talked about on Twitter about Theresa May and Boris Johnson the year before UK exits EU? 
          I am expecting a lot of similarities here, however I would like to refer to these for specific points in time to explain the                   sentiment analysis plot.

In [133]:
import pandas as pd

from pprint import pprint

# Gensim
import gensim
from gensim import models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

from gensim.models import Word2Vec

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','user', 'com', 'co', 'con', 'be', 'else', 'http', 'would','send', 
                   'do', 'try', 'tell', 'go', 'get', 'can', 'think', 'know', 'give', 'ask', 
               'next', 'find'])

In [134]:
#https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [135]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

### Getting the dataset

In [136]:
cdf = pd.read_csv('cdf.csv')

In [137]:
cdf['PM'].value_counts()/len(cdf)

none             0.901269
Boris Johnson    0.053743
Theresa May      0.044988
Name: PM, dtype: float64

In [138]:
#reducing dataset from 300k to 50k (removing 90% of the none values)
#cdf = cdf.drop(cdf[cdf['PM'] == 'none'].sample(frac=.98).index)
df = cdf.drop(cdf[(cdf.PM == 'none')].index)

In [139]:
df.dropna(inplace=True)

In [119]:
df['PM'].value_counts()

Boris Johnson    16407
Theresa May      13735
Name: PM, dtype: int64

In [140]:
#df_m = df[df['PM'] == 'Theresa May']
#df_b = df[df['PM'] == 'Boris Johnson']
#df_tm = df_m['TWEET_CLEANED'].tolist()
#df_bj = df_b['TWEET_CLEANED'].tolist()
df = df['TWEET_CLEANED'].tolist()

In [94]:
df

['nicola sturgeon tells boris johnson brexit offers democracy scotland',
 'ever brexit boris looking dodgy continental quilt still work',
 'media finally realised brexit disaster johnson artist ov',
 'borisjohnson better joke else tories toast brexit boris hints stay single market customs union 2021',
 'turbo charge brexit plans necessary funding prime minister boris johnson news kamaljoshi trend',
 'true democratic move would another referendum following questions brexit matter incl nodeal remain brexit borisjohnson deal brexit theresa may deal michaelgove jh4leader jeremycorbyn',
 'definitely without shadow doubt whatsoever believe swinson would better person trust corbyn definitely person consider corbyn dodgy johnson corbyn johnson want brexit',
 'guess boris going brexit restrictions brexit deal cal',
 'thoughts brexit cameron fucked country fucked expect fucked honestly excited boris think funny actually never know',
 'boris selected less hold highest office country inclusive peo

## Preparing the text

### Theresa May

### Creating a dictionary out of bigrams

In [141]:
bigram = gensim.models.Phrases(df, min_count=1, threshold=1) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [142]:
#remove Stop Words
data_words_nostops = remove_stopwords(df)
#creating bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
#Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'VERB', 'ADJ'])

KeyboardInterrupt: 

In [ ]:
print(data_lemmatized[:1][0][:30])

In [107]:
# Create Dictionary from the unique bigrams 
id2word = corpora.Dictionary(data_lemmatized)
id2word.filter_extremes(no_below=200)#, no_above=0.5) #excluding tokens that ocurred in less than 10 tweets and bigrams that occurred in more than 50% of the tweets
# Rebuild corpus based on the dictionary
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

### Using TF-IDF

#### A problem with this approach is that highly frequent words start to dominate in the document, but may not be representative to the model as less-frequent words. One way to fix this is to measure how unique (or how infrequent) a word is across all documents (or tweets), which is called the “inverse document frequency” or IDF. By introducing IDF, frequent words that are also frequent across all documents get penalized with less weight.
https://towardsdatascience.com/topic-modeling-of-2019-hr-tech-conference-twitter-d16cf75895b6

In [108]:
tfidf = models.TfidfModel(corpus)
tfidf_corpus = tfidf[corpus]

In [109]:
w2v_model = Word2Vec(min_count=100,
                     window=5,
                     size=100,
                     workers=4)

In [110]:
w2v_model.build_vocab(texts)         #building the vocabulary (from the tweets) for the model
w2v_model.train(texts, total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)
w2v_model.init_sims(replace=True)

In [111]:
w2v_model.wv.vocab

{'tell': <gensim.models.keyedvectors.Vocab at 0x144a64610>,
 'offer': <gensim.models.keyedvectors.Vocab at 0x144a64590>,
 'democracy': <gensim.models.keyedvectors.Vocab at 0x144a64ad0>,
 'look': <gensim.models.keyedvectors.Vocab at 0x144a64690>,
 'work': <gensim.models.keyedvectors.Vocab at 0x144a64c90>,
 'realise': <gensim.models.keyedvectors.Vocab at 0x144a7cc50>,
 'borisjohnson': <gensim.models.keyedvectors.Vocab at 0x144a7ca90>,
 'well': <gensim.models.keyedvectors.Vocab at 0x144a7ce10>,
 'tory': <gensim.models.keyedvectors.Vocab at 0x144a649d0>,
 'stay': <gensim.models.keyedvectors.Vocab at 0x144a64e50>,
 'single': <gensim.models.keyedvectors.Vocab at 0x144a7c090>,
 'market': <gensim.models.keyedvectors.Vocab at 0x144a7c250>,
 'plan': <gensim.models.keyedvectors.Vocab at 0x144a7c3d0>,
 'fund': <gensim.models.keyedvectors.Vocab at 0x144a7cbd0>,
 'true': <gensim.models.keyedvectors.Vocab at 0x144a7c190>,
 'democratic': <gensim.models.keyedvectors.Vocab at 0x144a7c690>,
 'move': <gen

In [129]:
w2v_model.wv.most_similar(positive=['theresa'])

[('process', 0.9717715978622437),
 ('negotiate', 0.9683218002319336),
 ('promise', 0.9655314683914185),
 ('negotiation', 0.957604169845581),
 ('action', 0.9570901989936829),
 ('negotiating', 0.9525808691978455),
 ('create', 0.9483357667922974),
 ('final', 0.9460707902908325),
 ('food', 0.9404304623603821),
 ('part', 0.9392752647399902)]

In [132]:
w2v_model.wv.most_similar(positive=['borisjohnson'])

[('class', 0.9969283938407898),
 ('fuck', 0.995520830154419),
 ('cause', 0.9937430024147034),
 ('go', 0.9930902123451233),
 ('wrong', 0.993037223815918),
 ('politician', 0.9929580688476562),
 ('other', 0.9920287132263184),
 ('live', 0.9916660785675049),
 ('hate', 0.9916154742240906),
 ('big', 0.9913168549537659)]

## Topic Modeling Via LDA

In [294]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)

In [277]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.494*"tory" + 0.162*"corbyn" + 0.084*"change" + 0.080*"brexiteer" + '
  '0.038*"person" + 0.030*"group" + 0.023*"young" + 0.022*"huge" + '
  '0.016*"nice" + 0.009*"picture"'),
 (1,
  '0.575*"people" + 0.071*"strategy" + 0.056*"place" + 0.049*"open" + '
  '0.043*"reason" + 0.042*"mind" + 0.040*"sure" + 0.019*"truth" + '
  '0.018*"medium" + 0.016*"sense"'),
 (2,
  '0.302*"election" + 0.228*"good" + 0.151*"british" + 0.077*"local" + '
  '0.065*"problem" + 0.051*"part" + 0.044*"control" + 0.037*"term" + '
  '0.011*"system" + 0.008*"previous"'),
 (3,
  '0.437*"party" + 0.151*"day" + 0.074*"democracy" + 0.063*"mess" + '
  '0.049*"life" + 0.043*"clean" + 0.035*"head" + 0.032*"street" + 0.028*"dead" '
  '+ 0.019*"guardian"'),
 (4,
  '0.345*"last" + 0.266*"week" + 0.189*"much" + 0.087*"threat" + '
  '0.034*"interesting" + 0.021*"information" + 0.000*"delay" + '
  '0.000*"conservative" + 0.000*"talk" + 0.000*"leader"'),
 (5,
  '0.178*"negotiation" + 0.157*"wrong" + 0.097*"mean" + 0.088*

In [278]:
for idx, topic in lda_model.show_topics(formatted=False, num_words= 10):
    print('Topic: {} \nWords: {}'.format(idx, '|'.join([w[0] for w in topic])))

Topic: 8 
Words: folk|storm|strength|attempt|immigration|door|mistake|planning|document|tough
Topic: 13 
Words: support|word|important|project|music|conservative|talk|delay|leader|cartoon
Topic: 10 
Words: month|article|may|family|several|fantasy|conservative|delay|leader|parliament
Topic: 19 
Words: work|high|point|thi|mother|quality|conservative|delay|talk|long
Topic: 12 
Words: government|money|one|access|conservative|delay|long|talk|parliament|company
Topic: 1 
Words: people|strategy|place|open|reason|mind|sure|truth|medium|sense
Topic: 2 
Words: election|good|british|local|problem|part|control|term|system|previous
Topic: 17 
Words: vote|labour|well|final|petition|sign|thank|effort|congratulation|conservative
Topic: 6 
Words: theresa|history|real|course|weak|modern|flip|conservative|delay|long
Topic: 16 
Words: brexit|deal|remainer|chief|crisis|police|former|food|staff|reality


In [247]:
'''
Topic 18: Crisis
Topic 29: Petition
Topic 12: Referendum
'''

'\nTopic 18: Crisis\nTopic 29: Petition\nTopic 12: Referendum\n'

### Checking Coherence Score

In [263]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.33488365719019264


In [253]:
top_words_per_topic = []
for t in range(lda_model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 10)])

df1 = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P'])

In [255]:
df1

,Topic,Word,P
0,0,head,0.472365
1,0,carry,0.159459
2,0,trash,0.084023
3,0,reply,0.049236
4,0,pointless,0.000150
...,...,...,...
295,29,conservative,0.000046
296,29,stop,0.000046
297,29,delay,0.000046
298,29,win,0.000046


In [242]:
lda_data =  pyLDAvis.gensim.prepare(lda_model, tfidf_corpus, id2word, mds='mmds')
pyLDAvis.display(lda_data)

/Users/ironhack/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
